# Data analysis
In below notebook, we analyze data and write some conclusions.

In [0]:
sas_token = "sp=racwdlmeop&st=2025-12-18T08:46:10Z&se=2025-12-20T17:01:10Z&spr=https&sv=2024-11-04&sr=c&sig=6zQKbMMlHY1NSoAhMXQ2dCgUcxjvUskDY5D4xibwKXA%3D"
storage_account_name = "newadbprojektkakastorage"
container_name = "data"

spark.conf.set(
    f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SAS"
)
spark.conf.set(
    f"fs.azure.sas.token.provider.type.{storage_account_name}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider"
)
spark.conf.set(
    f"fs.azure.sas.fixed.token.{storage_account_name}.dfs.core.windows.net",
    sas_token
)

base_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"
print(f"Skonfigurowano dostęp do: {base_path}")

## Analiza zbioru złączonego z df_flights i df_weather

In [0]:
file_path = "dbfs:/FileStore/tables/df_weather_jfk.csv"

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

print("Dane wczytane pomyślnie!")
display(df.limit(5))

df.printSchema()

In [0]:
df_bad_weather = df.filter(col("WEATHER_DELAY") > 0)
display(df_bad_weather)

Powyżej widać, że żaden lot nie został odwołany, gdy WEATHER_DELAY > 0. Możemy stwierdzić zatem, że WEATHER_DELAY dotyczy wyłącznie opóźnień.

In [0]:
from pyspark.sql.functions import min, max, col

df.select(
    min(col("ScheduledTimestamp")).alias("Najstarsza_Data"),
    max(col("ScheduledTimestamp")).alias("Najmlodsza_Data")
).display()

In [0]:
print(f"Liczba wszystkich lotów: {df.count()}")
print(f"Liczba opóźnionych lotów: {df.filter(col("DEPARTURE_DELAY")>0).count()}")
print(f"Liczba lotów, które wyleciały wcześniej niż powinny: {df.filter(col("DEPARTURE_DELAY")<0).count()}")

Loty, które wyleciały wcześniej zaburzają nam analizę wpływu pogody na opóźnienia.

In [0]:
df = df.filter(col("DEPARTURE_DELAY")>=0)
display(df.summary())

## Czy w któryś miesiąc opóźnienia są większe (wakacje, Boże Narodzenie? śnieg, mgła?)?


In [0]:
from pyspark.sql.functions import month, col, avg

df_grouped = df.withColumn("Month", month(col("ScheduledTimestamp"))) \
               .groupBy("Month") \
               .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
               .orderBy("Month")

display(df_grouped)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import month, col, avg

df_grouped = df_bad_weather.withColumn("Month", month(col("ScheduledTimestamp"))) \
               .groupBy("Month") \
               .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
               .orderBy("Month")

display(df_grouped)

Databricks visualization. Run in Databricks to view.

## Czy w weekend opóźnienia są większe?


In [0]:
from pyspark.sql.functions import when, col
df_features = df.withColumn("IsWeekend", when(col("DAY_OF_WEEK").isin([6, 7]), 1).otherwise(0))


display(df_features.groupBy("IsWeekend").agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")))

df_weekly_analysis = df_features.groupBy("DAY_OF_WEEK") \
    .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
    .orderBy("DAY_OF_WEEK")

display(df_weekly_analysis)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

## Czy loty długodystansowe są narażone na większe opóźnienia przy różnych warunkach atmosferycznych?


In [0]:
from pyspark.sql.functions import col, avg, floor

df_distance_bins = df.withColumn("Distance_Bin", floor(col("DISTANCE") / 100) * 100)

df_distance = df_distance_bins.groupBy("Distance_Bin") \
                         .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
                         .orderBy("Distance_Bin")

display(df_distance)

Databricks visualization. Run in Databricks to view.

## Czy któraś linia lotnicza ma większe opóźnienia niż pozostałe w złe warunki pogodowe?


In [0]:
from pyspark.sql.functions import col, avg, count, desc

airline_stats = df_bad_weather.groupBy("AIRLINE") \
    .agg(
        avg("WEATHER_DELAY").alias("Avg_Weather_Delay"),  # Średnie opóźnienie przez pogodę
        avg("DEPARTURE_DELAY").alias("Avg_Total_Delay"),  # Średnie całkowite opóźnienie (gdy była zła pogoda)
        count("*").alias("Flight_Count")                  # Liczba takich lotów 
    ) \
    .orderBy(col("Avg_Total_Delay").desc()) # Sortujemy od najgorszej

display(airline_stats)

airline_stats = df.groupBy("AIRLINE") \
    .agg(
        avg("DEPARTURE_DELAY").alias("Avg_Total_Delay"),  # Średnie całkowite opóźnienie (gdy była zła pogoda)
        count("*").alias("Flight_Count")                  # Liczba takich lotów 
    ) \
    .orderBy(col("Avg_Total_Delay").desc()) # Sortujemy od najgorszej

display(airline_stats)

## Wpływ prędkości wiatru na opóźnienia i odwołania lotów

In [0]:
from pyspark.sql.functions import round, avg, col, sum, count

df_wind_analysis = df.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(avg("DEPARTURE_DELAY").alias("AvgDelay"), count("*").alias("FlightCount")) \
    .orderBy("WindSpeed")

display(df_wind_analysis)

df_wind_analysis_cancel = df \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(
        avg("CANCELLED").alias("CancellationRate"),  # Zamiana 0/1 na procent (0.12 = 12%)
        sum("CANCELLED").alias("TotalCancelled"),    # Ile fizycznie odwołano
        count("*").alias("TotalFlights")             # Ile było wszystkich lotów przy tym wietrze
    ) \
    .filter(col("TotalFlights") > 10) \
    .orderBy("WindSpeed")

display(df_wind_analysis_cancel)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import round, avg, col

df_wind_analysis = df_bad_weather.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(avg("DEPARTURE_DELAY").alias("AvgDelay"), count("*").alias("FlightCount")) \
    .orderBy("WindSpeed")

display(df_wind_analysis)


df_wind_analysis = df_bad_weather \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(
        avg("CANCELLED").alias("CancellationRate"),  # Zamiana 0/1 na procent (0.12 = 12%)
        sum("CANCELLED").alias("TotalCancelled"),    # Ile fizycznie odwołano
        count("*").alias("TotalFlights")             # Ile było wszystkich lotów przy tym wietrze
    ) \
    .filter(col("TotalFlights") > 10) \
    .orderBy("WindSpeed")

display(df_wind_analysis_cancel)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

## Wpływ widoczności na opóźnienia i odwołania lotów

In [0]:
from pyspark.sql.functions import round, avg, col, count

df_vis_analysis = df.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYVISIBILITY"), 1).alias("Visibility")) \
    .agg(
        avg("DEPARTURE_DELAY").alias("AvgDelay"), 
        count("*").alias("FlightCount")
    ) \
    .filter(col("FlightCount") > 10) \
    .orderBy("Visibility")

display(df_vis_analysis)

Databricks visualization. Run in Databricks to view.

In [0]:
df_vis_analysis_cancel = df \
    .groupBy(round(col("HOURLYVISIBILITY")).alias("Visibility")) \
    .agg(
        avg("CANCELLED").alias("CancellationRate"),  # Zamiana 0/1 na procent (0.12 = 12%)
        sum("CANCELLED").alias("TotalCancelled"),    # Ile fizycznie odwołano
        count("*").alias("TotalFlights")             # Ile było wszystkich lotów przy tym wietrze
    ) \
    .filter(col("TotalFlights") > 5) \
    .orderBy("Visibility")

display(df_vis_analysis_cancel)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import round, avg, col, count

df_vis_analysis = df_bad_weather.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYVISIBILITY"), 1).alias("Visibility")) \
    .agg(
        avg("DEPARTURE_DELAY").alias("AvgDelay"), 
        count("*").alias("FlightCount")
    ) \
    .orderBy("Visibility")

display(df_vis_analysis)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import col, avg, hour

df_time_analysis = (
    df.filter(col("DEPARTURE_DELAY").isNotNull())
    .withColumn("Hour", hour(col("ScheduledTimestamp")))
    .groupBy("Hour")
    .agg(avg("DEPARTURE_DELAY").alias("AvgDelay"))
    .orderBy("Hour")
)

display(df_time_analysis)

Databricks visualization. Run in Databricks to view.